In [1]:
# !pip install openpyxl

In [2]:
import pandas as pd

In [3]:
df = pd.read_excel("traffic_stops.xlsx")

In [4]:
df.head()

,stop_date,stop_time,country_name,driver_gender,driver_age_raw,driver_age,driver_race,violation_raw,violation,search_conducted,search_type,stop_outcome,is_arrested,stop_duration,drugs_related_stop,vehicle_number
0,2020-01-01,00:00:00,Canada,M,59,19,Asian,Drunk Driving,Speeding,True,Vehicle Search,Ticket,True,16-30 Min,True,UP76DY3473
1,2020-01-01,00:01:00,India,M,35,58,Other,Other,Other,False,Vehicle Search,Arrest,True,16-30 Min,True,RJ83PZ4441
2,2020-01-01,00:02:00,USA,M,26,76,Black,Signal Violation,Speeding,False,Frisk,Ticket,True,16-30 Min,True,RJ32OM7264
3,2020-01-01,00:03:00,Canada,M,26,76,Black,Speeding,DUI,True,Frisk,Warning,False,0-15 Min,True,RJ76TI3807
4,2020-01-01,00:04:00,Canada,M,62,75,Other,Speeding,Other,False,Vehicle Search,Arrest,True,16-30 Min,False,WB63BB8305


In [5]:
print(df.isnull().sum())

stop_date                 0
stop_time                 0
country_name              0
driver_gender             0
driver_age_raw            0
driver_age                0
driver_race               0
violation_raw             0
violation                 0
search_conducted          0
search_type           21720
stop_outcome              0
is_arrested               0
stop_duration             0
drugs_related_stop        0
vehicle_number            0
dtype: int64


In [7]:
df_bfill = df.bfill()
df_ffill = df.ffill()
df_mean = df.fillna(df.mean(numeric_only=True))
df_mode = df.fillna(df.mode().iloc[0])
df_zero = df.fillna(0)
df_dropped = df.dropna()

In [8]:
print("Null values after using bfill:\n", df_bfill.isnull().sum())
print("Null values after using mean:\n", df_mean.isnull().sum())
print("Null values after using mode:\n", df_mode.isnull().sum())
print("Null values after using 0:\n", df_zero.isnull().sum())
print("Shape after dropping rows with null values:", df_dropped.shape)

Null values after using bfill:
 stop_date             0
stop_time             0
country_name          0
driver_gender         0
driver_age_raw        0
driver_age            0
driver_race           0
violation_raw         0
violation             0
search_conducted      0
search_type           2
stop_outcome          0
is_arrested           0
stop_duration         0
drugs_related_stop    0
vehicle_number        0
dtype: int64
Null values after using mean:
 stop_date                 0
stop_time                 0
country_name              0
driver_gender             0
driver_age_raw            0
driver_age                0
driver_race               0
violation_raw             0
violation                 0
search_conducted          0
search_type           21720
stop_outcome              0
is_arrested               0
stop_duration             0
drugs_related_stop        0
vehicle_number            0
dtype: int64
Null values after using mode:
 stop_date             0
stop_time             0


In [9]:
df['search_type'] = (
    df.groupby(['country_name', 'driver_race'])['search_type']
      .transform(lambda x: x.fillna(x.mode()[0] if not x.mode().empty else 'Unknown'))
)

In [10]:
print("Remaining null values in search_type:", df['search_type'].isnull().sum())

Remaining null values in search_type: 0


In [11]:
print(df.isnull().sum())

stop_date             0
stop_time             0
country_name          0
driver_gender         0
driver_age_raw        0
driver_age            0
driver_race           0
violation_raw         0
violation             0
search_conducted      0
search_type           0
stop_outcome          0
is_arrested           0
stop_duration         0
drugs_related_stop    0
vehicle_number        0
dtype: int64


In [12]:
df.head()

,stop_date,stop_time,country_name,driver_gender,driver_age_raw,driver_age,driver_race,violation_raw,violation,search_conducted,search_type,stop_outcome,is_arrested,stop_duration,drugs_related_stop,vehicle_number
0,2020-01-01,00:00:00,Canada,M,59,19,Asian,Drunk Driving,Speeding,True,Vehicle Search,Ticket,True,16-30 Min,True,UP76DY3473
1,2020-01-01,00:01:00,India,M,35,58,Other,Other,Other,False,Vehicle Search,Arrest,True,16-30 Min,True,RJ83PZ4441
2,2020-01-01,00:02:00,USA,M,26,76,Black,Signal Violation,Speeding,False,Frisk,Ticket,True,16-30 Min,True,RJ32OM7264
3,2020-01-01,00:03:00,Canada,M,26,76,Black,Speeding,DUI,True,Frisk,Warning,False,0-15 Min,True,RJ76TI3807
4,2020-01-01,00:04:00,Canada,M,62,75,Other,Speeding,Other,False,Vehicle Search,Arrest,True,16-30 Min,False,WB63BB8305


In [18]:
import mysql.connector

conn_mysql = mysql.connector.connect(
    host="localhost",
    user="root",
    password="Jan@9302",
    database="traffic"
)
cursor_mysql = conn_mysql.cursor()
print("MySQL connection established!")

MySQL connection established!


In [19]:
cursor_mysql.execute("CREATE DATABASE IF NOT EXISTS traffic;")
print("MySQL database 'traffic' created successfully!")

MySQL database 'traffic' created successfully!


In [20]:
create_table_query = """
CREATE TABLE IF NOT EXISTS traffic_data (
    stop_date DATE,
    stop_time TIME,
    country_name VARCHAR(100),
    driver_gender VARCHAR(50),
    driver_age_raw VARCHAR(50),
    driver_age INT,
    driver_race VARCHAR(50),
    violation_raw VARCHAR(255),
    violation VARCHAR(255),
    search_conducted VARCHAR(50),
    search_type VARCHAR(100),
    stop_outcome VARCHAR(100),
    is_arrested VARCHAR(5),
    stop_duration VARCHAR(50),
    drugs_related_stop VARCHAR(5),
    vehicle_number VARCHAR(50),
    KEY idx_date (stop_date),
    KEY idx_country (country_name),
    KEY idx_violation (violation)
);
"""

cursor_mysql.execute(create_table_query)
conn_mysql.commit()
print("Table 'traffic_db' created successfully!")

Table 'traffic_db' created successfully!


In [21]:
data_list = df.values.tolist()
query = """
INSERT INTO traffic_data (
    stop_date, stop_time, country_name, driver_gender,
    driver_age_raw, driver_age, driver_race, violation_raw,
    violation, search_conducted, search_type, stop_outcome,
    is_arrested, stop_duration, drugs_related_stop, vehicle_number
) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""
cursor_mysql.executemany(insert_query, data_list)
conn_mysql.commit()
print(f"{len(data_list)} rows inserted successfully!")
cursor_mysql.close()
conn_mysql.close()

65538 rows inserted successfully!
